<a href="https://colab.research.google.com/github/ormarin2000/Portfolio/blob/main/BB_Volatility_Scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
# Done in colab
!pip install ta

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from ta import add_all_ta_features
import yfinance as yf
from ta.volatility import BollingerBands

In [ ]:
# We will scan just for stocks in the S&P500
# Get tickers in the S&P500
tnames = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers_names = tnames.Symbol.to_list()
tickers_names.sort()
# We change the format to be compatible
tickers_names[tickers_names.index('BF.B')] = 'BF-B'
tickers_names[tickers_names.index('BRK.B')] = 'BRK-B'

In [ ]:
# Original scanner, tends to not return many tickers because of the amount of conditions.
# Conditions:
# Bands width under 5% of its 6-month low
# Bands range under 5% of its 6-month low
# Volume within 5% or more of its 6-month high
# Upper or lower cross depending on the position
# 52-week support or resistance break depending on the position

long_scanner = []
short_scanner = []
for i in tickers_names:

  # Get the data for tickers from yahoo finance
  ticker = yf.download(i , start = '2014-03-31', auto_adjust= True, interval = '1wk', progress = False)
  pd.DataFrame(ticker)

  # Initialize bollinger bands indicator
  indicator_bb = BollingerBands(close=ticker.Close, window=20, window_dev=2)
  # Add Bollinger bands features
  ticker['bb_bbm'] = indicator_bb.bollinger_mavg()
  ticker['bb_bbh'] = indicator_bb.bollinger_hband()
  ticker['bb_bbl'] = indicator_bb.bollinger_lband()

  # Bollinger band width
  ticker['bbw'] = (ticker.bb_bbh-ticker.bb_bbl)/ticker.bb_bbm
  ticker['bbw_lowest_six_months'] = ticker.bbw.rolling(26).min()
  # 5% for 95% confidence interval
  ticker['six_month_low_distance_checker'] = ticker.bbw < (ticker.bbw_lowest_six_months*1.05)
  # Add NaN where it corresponds instead of just False
  ticker.loc[(ticker.bbw.isnull()) | (ticker.bbw_lowest_six_months.isnull()), 'six_month_low_distance_checker'] = np.nan

  # Top and bottom bollinger bands range
  ticker['range'] = ticker.bb_bbh - ticker.bb_bbl
  # Average of last 6 months
  ticker['range_lowest'] = ticker.range.rolling(26).min()
  # Less than 50% of average 6-month range
  ticker['six_month_range_low_checker'] = ticker.range < (ticker.range_lowest*1.05)
  # Add NaN where it corresponds instead of just False
  ticker.loc[(ticker.range.isnull()) | (ticker.range_lowest.isnull()), 'six_month_range_low_checker'] = np.nan

  # Six-month volume
  ticker['volume_max_six_months'] = ticker.Volume.rolling(26).max()
  ticker['six_month_max_volume_checker'] = ticker.Volume > (ticker.volume_max_six_months*0.95)
  # Add NaN where it corresponds instead of just False
  ticker.loc[(ticker.volume_max_six_months.isnull()), 'six_month_max_volume_checker'] = np.nan

  # Bollinger cross
  ticker['upper_cross'] = ticker.Close > ticker.bb_bbh
  ticker['lower_cross'] = ticker.Close < ticker.bb_bbl
  # Add NaN where it corresponds instead of just False
  ticker.loc[(ticker.bb_bbh.isnull()), 'upper_cross'] = np.nan
  ticker.loc[(ticker.bb_bbl.isnull()), 'lower_cross'] = np.nan

  # Support and resistance break
  # 52-week support and resistance
  ticker['support'] = ticker.Close.rolling(52).max()
  ticker['resistance'] = ticker.Close.rolling(52).min()
  ticker['support_break'] = ((ticker.Close.shift(1) > ticker.support.shift(1)) & (ticker.Close < ticker.support))
  ticker['resistance_break'] = ((ticker.Close.shift(1) < ticker.resistance.shift(1)) & (ticker.Close > ticker.resistance))
  # Add NaN where it corresponds instead of just False
  ticker.loc[(ticker.support.isnull()), 'support_break'] = np.nan
  ticker.loc[(ticker.resistance.isnull()), 'resistance_break'] = np.nan

  # Signals
  ticker['long_signal'] = ticker.six_month_low_distance_checker & ticker.six_month_range_low_checker \
  & ticker.six_month_max_volume_checker & ticker.upper_cross & ticker.resistance_break
  ticker['short_signal'] = ticker.six_month_low_distance_checker & ticker.six_month_range_low_checker \
  & ticker.six_month_max_volume_checker & ticker.lower_cross & ticker.support_break

  # Scanner
  if ticker.long_signal.iloc[-1] == True:
    long_scanner.append(i)
  elif ticker.short_signal.iloc[-1] == True:
    short_scanner.append(i)

In [ ]:
# Visualize scanner
while True:
  try:
    choose = int(input("""
    Long or Short Positions?
    [1]. Long
    [2]. Short
    """))
    if choose < 1 or choose > 2:
      raise ValueError
    elif choose == 1:
      print('Long stocks in the S&P500: ', long_scanner)
    elif choose == 2:
      print('Short stocks in the S&P500: ', short_scanner)
    break
  except ValueError:
    print('Write 1 for Long Positions and 2 for Short Positions')


    Long or Short Positions?
    [1]. Long
    [2]. Short
    2
Short stocks in the S&P500:  []


In [ ]:
# Adjustment for getting a wider variety of stocks
# Conditions:
# Bands range under 5% of its 6-month low
# Volume within 5% or more of its 6-month high
# Upper or lower cross depending on the position
# 52-week support or resistance break depending on the position

long_scanner = []
short_scanner = []
for i in tickers_names:

  # Get the data for tickers from yahoo finance
  ticker = yf.download(i , start = '2014-03-31', auto_adjust= True, interval = '1wk', progress = False)
  pd.DataFrame(ticker)

  # Initialize bollinger bands indicator
  indicator_bb = BollingerBands(close=ticker.Close, window=20, window_dev=2)
  # Add bollinger bands features
  ticker['bb_bbm'] = indicator_bb.bollinger_mavg()
  ticker['bb_bbh'] = indicator_bb.bollinger_hband()
  ticker['bb_bbl'] = indicator_bb.bollinger_lband()

  # Bollinger band width
  ticker['bbw'] = (ticker.bb_bbh-ticker.bb_bbl)/ticker.bb_bbm
  ticker['bbw_lowest_six_months'] = ticker.bbw.rolling(26).min()
  # 5% for 95% confidence interval
  ticker['six_month_low_distance_checker'] = ticker.bbw < (ticker.bbw_lowest_six_months*1.05)
  # Add NaN where it corresponds instead of just False
  ticker.loc[(ticker.bbw.isnull()) | (ticker.bbw_lowest_six_months.isnull()), 'six_month_low_distance_checker'] = np.nan

  # Top and bottom bollinger bands range
  ticker['range'] = ticker.bb_bbh - ticker.bb_bbl
  # Average of last 6 months
  ticker['range_lowest'] = ticker.range.rolling(26).min()
  # Less than 50% of average 6-month range
  ticker['six_month_range_low_checker'] = ticker.range < (ticker.range_lowest*1.05)
  # Add NaN where it corresponds instead of just False
  ticker.loc[(ticker.range.isnull()) | (ticker.range_lowest.isnull()), 'six_month_range_low_checker'] = np.nan

  # Six-month volume
  ticker['volume_max_six_months'] = ticker.Volume.rolling(26).max()
  ticker['six_month_max_volume_checker'] = ticker.Volume > (ticker.volume_max_six_months*0.95)
  # Add NaN where it corresponds instead of just False
  ticker.loc[(ticker.volume_max_six_months.isnull()), 'six_month_max_volume_checker'] = np.nan

  # Bollinger cross
  ticker['upper_cross'] = ticker.Close > ticker.bb_bbh
  ticker['lower_cross'] = ticker.Close < ticker.bb_bbl
  # Add NaN where it corresponds instead of just False
  ticker.loc[(ticker.bb_bbh.isnull()), 'upper_cross'] = np.nan
  ticker.loc[(ticker.bb_bbl.isnull()), 'lower_cross'] = np.nan

  # Support and resistance break
  # 52-week support and resistance
  ticker['support'] = ticker.Close.rolling(52).max()
  ticker['resistance'] = ticker.Close.rolling(52).min()
  ticker['support_break'] = ((ticker.Close.shift(1) > ticker.support.shift(1)) & (ticker.Close < ticker.support))
  ticker['resistance_break'] = ((ticker.Close.shift(1) < ticker.resistance.shift(1)) & (ticker.Close > ticker.resistance))
  # Add NaN where it corresponds instead of just False
  ticker.loc[(ticker.support.isnull()), 'support_break'] = np.nan
  ticker.loc[(ticker.resistance.isnull()), 'resistance_break'] = np.nan

  # Signals
  ticker['long_signal'] = ticker.six_month_range_low_checker \
  & ticker.six_month_max_volume_checker & ticker.upper_cross & ticker.resistance_break
  ticker['short_signal'] = ticker.six_month_range_low_checker \
  & ticker.six_month_max_volume_checker & ticker.lower_cross & ticker.support_break

  # Scanner
  if ticker.long_signal.iloc[-1] == True:
    long_scanner.append(i)
  elif ticker.short_signal.iloc[-1] == True:
    short_scanner.append(i)

In [ ]:
# Visualize scanner
while True:
  try:
    choose = int(input("""
    Long or Short Positions?
    [1]. Long
    [2]. Short
    """))
    if choose < 1 or choose > 2:
      raise ValueError
    elif choose == 1:
      print('Long stocks in the S&P500: ', long_scanner)
    elif choose == 2:
      print('Short stocks in the S&P500: ', short_scanner)
    break
  except ValueError:
    print('Write 1 for Long Positions and 2 for Short Positions')


    Long or Short Positions?
    [1]. Long
    [2]. Short
    2
Short stocks in the S&P500:  []


In [ ]:
# Removing support and resistance breaks
# Conditions:
# Bands range under 5% of its 6-month low
# Volume within 5% or more of its 6-month high
# Upper or lower cross depending on the position

long_scanner = []
short_scanner = []
for i in tickers_names:

  # Get the data for tickers from yahoo finance
  ticker = yf.download(i , start = '2014-03-31', auto_adjust= True, interval = '1wk', progress = False)
  pd.DataFrame(ticker)

  # Initialize bollinger bands indicator
  indicator_bb = BollingerBands(close=ticker.Close, window=20, window_dev=2)
  # Add bollinger bands features
  ticker['bb_bbm'] = indicator_bb.bollinger_mavg()
  ticker['bb_bbh'] = indicator_bb.bollinger_hband()
  ticker['bb_bbl'] = indicator_bb.bollinger_lband()

  # Bollinger band width
  ticker['bbw'] = (ticker.bb_bbh-ticker.bb_bbl)/ticker.bb_bbm
  ticker['bbw_lowest_six_months'] = ticker.bbw.rolling(26).min()
  # 5% for 95% confidence interval
  ticker['six_month_low_distance_checker'] = ticker.bbw < (ticker.bbw_lowest_six_months*1.05)
  # Add NaN where it corresponds instead of just False
  ticker.loc[(ticker.bbw.isnull()) | (ticker.bbw_lowest_six_months.isnull()), 'six_month_low_distance_checker'] = np.nan

  # Top and bottom bollinger bands range
  ticker['range'] = ticker.bb_bbh - ticker.bb_bbl
  # Average of last 6 months
  ticker['range_lowest'] = ticker.range.rolling(26).min()
  # Less than 50% of average 6-month range
  ticker['six_month_range_low_checker'] = ticker.range < (ticker.range_lowest*1.05)
  # Add NaN where it corresponds instead of just False
  ticker.loc[(ticker.range.isnull()) | (ticker.range_lowest.isnull()), 'six_month_range_low_checker'] = np.nan

  # Six-month volume
  ticker['volume_max_six_months'] = ticker.Volume.rolling(26).max()
  ticker['six_month_max_volume_checker'] = ticker.Volume > (ticker.volume_max_six_months*0.95)
  # Add NaN where it corresponds instead of just False
  ticker.loc[(ticker.volume_max_six_months.isnull()), 'six_month_max_volume_checker'] = np.nan

  # Bollinger cross
  ticker['upper_cross'] = ticker.Close > ticker.bb_bbh
  ticker['lower_cross'] = ticker.Close < ticker.bb_bbl
  # Add NaN where it corresponds instead of just False
  ticker.loc[(ticker.bb_bbh.isnull()), 'upper_cross'] = np.nan
  ticker.loc[(ticker.bb_bbl.isnull()), 'lower_cross'] = np.nan

  # Support and resistance break
  # 52-week support and resistance
  ticker['support'] = ticker.Close.rolling(52).max()
  ticker['resistance'] = ticker.Close.rolling(52).min()
  ticker['support_break'] = ((ticker.Close.shift(1) > ticker.support.shift(1)) & (ticker.Close < ticker.support))
  ticker['resistance_break'] = ((ticker.Close.shift(1) < ticker.resistance.shift(1)) & (ticker.Close > ticker.resistance))
  # Add NaN where it corresponds instead of just False
  ticker.loc[(ticker.support.isnull()), 'support_break'] = np.nan
  ticker.loc[(ticker.resistance.isnull()), 'resistance_break'] = np.nan

  # Signals
  ticker['long_signal'] = ticker.six_month_range_low_checker \
  & ticker.six_month_max_volume_checker & ticker.upper_cross
  ticker['short_signal'] = ticker.six_month_range_low_checker \
  & ticker.six_month_max_volume_checker & ticker.lower_cross

  # Scanner
  if ticker.long_signal.iloc[-1] == True:
    long_scanner.append(i)
  elif ticker.short_signal.iloc[-1] == True:
    short_scanner.append(i)

In [ ]:
# Visualize scanner
while True:
  try:
    choose = int(input("""
    Long or Short Positions?
    [1]. Long
    [2]. Short
    """))
    if choose < 1 or choose > 2:
      raise ValueError
    elif choose == 1:
      print('Long stocks in the S&P500: ', long_scanner)
    elif choose == 2:
      print('Short stocks in the S&P500: ', short_scanner)
    break
  except ValueError:
    print('Write 1 for Long Positions and 2 for Short Positions')


    Long or Short Positions?
    [1]. Long
    [2]. Short
    2
Short stocks in the S&P500:  []
